In [1]:
import numpy as np
from random import random
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue
from collections import namedtuple

In [23]:
PROBLEM_SIZE = 5
NUM_SETS = 10
# Arrays where all elements have a probability of 20% of being true (Basic building blocks to cover the set)
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [3]:
def goal_check(state : State) :
    # Combination of every chosen blocks and checking if all is covered --> Test for goal state
    return np.all(reduce(np.logical_or, (SETS[i] for i in state[0]), np.array([False for _ in range(PROBLEM_SIZE)])))

In [24]:
# If the whole set can't be covered even with all the building sets, the problem can't be solved
assert goal_check(State(set(range(NUM_SETS)),set())), "Problem not solvable"

Solving the problem with uninformed strategy

In [33]:
# Change PriorityQueue with SimpleQueue to have Breadth-first, and LifoQueue to have Depth-first
frontier = SimpleQueue()
# Initial state
frontier.put(State(set(), set(range(NUM_SETS))))

current_state = frontier.get()
counter = 0
while not goal_check(current_state) : 
    counter += 1
    for a in current_state[1] :
        new_state = State(current_state[0] ^ {a}, current_state[1] ^ {a})
        frontier.put(new_state)
    current_state = frontier.get()
    
print("Solved in", counter, "steps :", current_state.taken)

Solved in 114 steps : {0, 2, 7}


Solving the problem with A*

In [6]:
def h(state : State, set_size) :
    return set_size - np.sum(reduce(np.logical_or, (SETS[i] for i in state.taken), np.array([False for _ in range(set_size)])))

In [21]:
def g(state : State) :
    return len(state.taken)

In [9]:
stateTest = State({0,2}, {1,3,4,5,6,7,8,9}) 
h(stateTest, PROBLEM_SIZE)
g(stateTest)

2

In [34]:
frontier = PriorityQueue()

cur_state = State(set(), set([i for i in range(NUM_SETS)]))
# frontier.put((0,cur_state))

counter_steps = 0
while cur_state is not None and not goal_check(cur_state) :
    counter_steps += 1
    for action in cur_state.not_taken :
        new_state = State(cur_state.taken ^ {action}, cur_state.not_taken ^ {action})
        new_cost = g(new_state) + h(new_state, PROBLEM_SIZE)
        frontier.put((new_cost, new_state))
    _, cur_state = frontier.get()

if cur_state is None :
    print("No solution was found :(")
else :
    print("Solution found :", cur_state.taken,"in", counter_steps, "steps")

Solution found : {0, 4, 7} in 6 steps
